In [22]:
import numpy as np

In [23]:
# Funcion para calcular el proximo tiempo de llegada
def getTs(t,lamda_max,homogeneo = True):
    if homogeneo:
        return t - (np.log(np.random.uniform())/lamda_max)
    else:
        while True:
            t = t - (np.log(np.random.uniform())/lamda_max)
            if np.random.uniform() <= np.random.poisson(t)/lamda_max:
                return t

In [70]:
def simular(duracion_simulacion = 100,lamda_max = 5000,velocidad = 100,no_cpus = 2):
    # VARIABLES DE SIMULACION
    t = 0 # tiempo de la simulacion
    n = 0 # numero de clientes en el sistema
    
    # Variables contadores
    Na = 0 # Numero de llegadas
    C = np.zeros(no_cpus) # Numero de salidas en el i-esimo servidor
    Ocupado = np.zeros(no_cpus) # Tiempo que estubo ocupado el servidor
    
    # Variables de salidas
    A = {} # Tiempo de la i-esima llegada
    D = {} # Tiempo de la i-esima salida
    NT = [] # Tiempos de cada cliente en espera
    
    # Variables de eventos
    ta = getTs(t,lamda_max) # Tiempo de la proxima llegada
    td = np.zeros(no_cpus) + np.infty # tiempo de salida del i-esimo servidor
    ids = np.zeros(no_cpus) # numero del cliente atendido en el i-esimo servidor 
    
    # SIMULACION
    while t < duracion_simulacion:
        # Si el tiempo de llegada es el menor tiempo en el sistema,
        # es decir, que hay un servidor disponible, entonces
        # atendemos al cliente
        if ta == min(ta,min(td)): 
            t = ta # avanzamos el tiempo al tiempo de llegada
            Na += 1 # Contamos una llegada mas
            ta = getTs(t,lamda_max) # calculamos el proximo tiempo de llegada
            A[Na] = t # guardamos el tiempo de la Na-esima llegada
            if n < no_cpus: # si hay menos clientes que servidores
                for i in range(no_cpus):
                    if ids[i] == 0: # Si el servidor esta disponible
                        ids[i] = Na # atendemos al Na-esimo cliente
                        NT = np.append(NT,t - A[Na])
                        td[i] = t + np.random.exponential(velocidad) # calculamos su tiempo de salida
                        Ocupado[i] += td[i]-t # Calculamos el tiempo que va a estar en el servidor
                        break;
            n += 1 # Contamos al nuevo cliente en el sistema
        else:
            ti = np.argmin(td) # Escogemos al servidor que se desocupa primero
            t = td[ti] # avanzamos el tiempo al tiempo en que se desocupa
            C[ti] += 1 # Contamos que el i-esimo servidor atendio un cliente mas
            D[ids[ti]] = t # Contamos el tiempo de la i-esima salida
            if n <= no_cpus: # Si hay menos o igual clientes que servidores
                ids[ti] = 0 # Despachamos al cliente del servidor a desocupar
                td[ti] = np.infty # Liberamos al servidor
            else: # Si todavia hay mas clientes por atender
                index = max(ids) + 1
                ids[ti] = index # Le asignamos al servidor libre al nuevo cliente
                NT = np.append(NT , t - A[index])
                td[ti] = t + np.random.exponential(velocidad) # Calculamos el tiempo de salida
                Ocupado[ti] += td[ti]-t # Calculamos el tiempo que va a estar en el servidor
            n -= 1 # Descontamos al cliente atendido del sistema
        
        # RESULTADOS
        # Retornamos un objeto con los siguientes resultados:
        # Na: numero de llegadas
        # Nd: numero de salidas
        # n: numero de clientes en el sistema al final de la simulacion
        # ta: tiempo de la siguiente llegada al final de la simulacion
        # C: cantidad de clientes atendidos por cada servidor
        # td: tiempos de salida para cada servidor al final de la simulacion
        # A: llegadas y sus tiempos
        # D: salidas y sus tiempos
        # Ocupado: tiempo que paso ocupado el servidor
        # NT: tiempo en espera
    return {"Na": Na, "Nd":sum(C), "n":n, "ta": ta, "C": C, "td":td, "A": A, "D": D, "Ocupado": Ocupado, "NT": NT}

In [83]:
max_solicitudes = [2400,6000]
duraciones = [60]
Proveedores = [
    {
        "nombre": "Gorilla Megacomputing",
        "no_cpus": 1,
        "velocidad": 1/(100*60)
    },
    {
        "nombre": "Ants smart computing",
        "no_cpus": 10,
        "velocidad": 1/(10*60)
    },
]

In [84]:
for duracion_simulacion in duraciones:
    print("Duracion simulacion: ",duracion_simulacion)
    for lamda_max in max_solicitudes:
        print(" - Maximas solicitudes: ",lamda_max)
        for proveedor in Proveedores:
            resultados = simular(duracion_simulacion,lamda_max,proveedor["velocidad"],proveedor["no_cpus"])
            print("\t",proveedor["nombre"],", CPUS: ",proveedor["no_cpus"],", Velocidad:",np.round(proveedor["velocidad"],5),"/s")
            print("\tSolicitudes atendidas: ",resultados["C"])
            print("\tTiempos de ocupacion de servidores: ", resultados["Ocupado"])
            print("\tTiempos libres de servidores: ",np.maximum(np.ones(proveedor["no_cpus"])*duracion_simulacion - resultados["Ocupado"],0))
            print("\tTiempo de solicitudes en cola: ",np.round(sum(resultados["NT"]),5))
            print("\tPromedio de solicitudes en cola:",np.round(np.mean(resultados["NT"]),5))
            print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados["td"][-1],8))
            print("")
        print("")
    print("")

Duracion simulacion:  60
 - Maximas solicitudes:  2400
	 Gorilla Megacomputing , CPUS:  1 , Velocidad: 0.00017 /s
	Solicitudes atendidas:  [144032.]
	Tiempos de ocupacion de servidores:  [23.86975369]
	Tiempos libres de servidores:  [36.13024631]
	Tiempo de solicitudes en cola:  16.00097
	Promedio de solicitudes en cola: 0.00011
	Tiempo de salida de la ultima solicitud atendida:  60.0001755

	 Ants smart computing , CPUS:  10 , Velocidad: 0.00167 /s
	Solicitudes atendidas:  [28974. 26525. 23771. 20228. 16159. 11758.  7921.  4744.  2553.  1288.]
	Tiempos de ocupacion de servidores:  [48.03546898 44.33630121 39.49019224 33.46964929 26.87444856 19.76462829
 13.33762571  7.98248566  4.23589471  2.22130508]
	Tiempos libres de servidores:  [11.96453102 15.66369879 20.50980776 26.53035071 33.12555144 40.23537171
 46.66237429 52.01751434 55.76410529 57.77869492]
	Tiempo de solicitudes en cola:  0.34494
	Promedio de solicitudes en cola: 0.0
	Tiempo de salida de la ultima solicitud atendida:  in

### Notas:
- Se recomienda alquilar 20 servidores para asegurar que no hayan clientes en cola, pero depende de las solicitudes maximas eperadas
- Se recomienda alquilar los 10 servidores de Ants Smart computig, porque producen un tiempo menor de espera. Dependiendo del presupuesto se recomienda alquilar de 15 a 20 servidores para evitar o disminuir tiempos de espera